In [1]:
import pandas as pd

In [2]:
import os
os.getcwd()


'/workspaces/Currency-speculation'

In [3]:
# Define the path to the Excel file
xls = "/workspaces/Currency-speculation/Three factor mode (1).xlsx"


Inflation_raw = pd.read_excel(xls, sheet_name='CPI')
Interest_Raw = pd.read_excel(xls, sheet_name='Interest rate')
Current_account_raw = pd.read_excel(xls, sheet_name='current account balance')





In [4]:
#dropping the first row
Inflation_raw = Inflation_raw.drop(Inflation_raw.index[0])
Interest_Raw = Interest_Raw.drop(Interest_Raw.index[0])
Current_account_raw = Current_account_raw.drop(Current_account_raw.index[0])
# Resetting the index
Inflation_raw.reset_index(drop=True, inplace=True)
Interest_Raw.reset_index(drop=True, inplace=True)
Current_account_raw.reset_index(drop=True, inplace=True)


In [5]:
#changing the data type of the first column to date by indexing the first column
Inflation_raw['Unnamed: 0'] = pd.to_datetime(Inflation_raw['Unnamed: 0'], format='%Y-%m')
Interest_Raw['Unnamed: 0'] = pd.to_datetime(Interest_Raw['Unnamed: 0'], format='%Y-%m')
Current_account_raw['Unnamed: 0'] = pd.to_datetime(Current_account_raw['Unnamed: 0'], format='%Y-%m')

#renaming the first column to 'Date'
Inflation_raw.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
Interest_Raw.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
Current_account_raw.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)




In [6]:
#converting all the columns to numeric type
def convert_to_numeric(df):
    for col in df.columns[1:]:  # Skip the first column (Date)
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return df
Inflation_raw = convert_to_numeric(Inflation_raw)
Interest_Raw = convert_to_numeric(Interest_Raw)
Current_account_raw = convert_to_numeric(Current_account_raw)



In [7]:
#Creating three empty dataframes to store the differentials
Inflation_diff = pd.DataFrame()
Interest_diff = pd.DataFrame()
Current_account_diff = pd.DataFrame()


In [8]:
#calculating the inflation differential
Inflation_raw['USDJPY'] = Inflation_raw['US']-Inflation_raw['UK']


In [9]:
#function to calculate all the differentials
countries = ['US','IN', 'JPN','UK','EU', 'SZR']
def calculate_differentials(df, df_new):
        for j in range(1, len(countries)):
            country1 = countries[0]
            country2 = countries[j]
            swap_name = f"{country1}{country2}"
            df_new[swap_name] = df[country1] - df[country2]
        df_new['Date'] = df['Date']
        df_new.set_index('Date', inplace=True)
        return df_new
calculate_differentials(Inflation_raw, Inflation_diff)
calculate_differentials(Interest_Raw, Interest_diff)
calculate_differentials(Current_account_raw, Current_account_diff)



,USIN,USJPN,USUK,USEU,USSZR
Date,,,,,
2006-01-31,-4.49,-9.30,-3.34,-5.24,-16.91
2006-02-28,-4.49,-9.30,-3.34,-5.24,-16.91
2006-03-31,-4.49,-9.30,-3.34,-5.24,-16.91
2006-04-28,-4.49,-9.30,-3.34,-5.24,-16.91
2006-05-31,-4.49,-9.30,-3.34,-5.24,-16.91
...,...,...,...,...,...
2025-03-31,-4.00,-9.52,-1.71,-6.98,-10.82
2025-04-30,-4.00,-9.52,-1.71,-6.98,-10.82
2025-05-30,-4.00,-9.52,-1.71,-6.98,-10.82


In [10]:
#function to round all the values in the dataframe to 2 decimal places
def round_dataframe(df):
    for col in df.columns:

        if not col == 'Date':
            df = df.round(2)
    return df
Inflation_diff = round_dataframe(Inflation_diff)
Interest_diff = round_dataframe(Interest_diff)
Current_account_diff = round_dataframe(Current_account_diff)



In [11]:
#renaming the swaps
def rename_swaps(df):
    df.rename(columns={
        'USIN': 'USDINR',
        'USJPN': 'USDJPY',
        'USUK': 'USDGBP',
        'USEU': 'USDEUR',
        'USSZR': 'USDCHF',
    }, inplace=True)

rename_swaps(Inflation_diff)
rename_swaps(Interest_diff)
rename_swaps(Current_account_diff)


In [12]:
#adding an index to the dataframes by converting it back to string format
Inflation_diff.index = Inflation_diff.index.strftime('%Y-%m')
Interest_diff.index = Interest_diff.index.strftime('%Y-%m')
Current_account_diff.index = Current_account_diff.index.strftime('%Y-%m')


In [13]:
#downloading the dataframes as csv files
Inflation_diff.to_csv('Inflation_diff.csv')
Interest_diff.to_csv('Interest_diff.csv')
Current_account_diff.to_csv('Current_account_diff.csv')
